<a href="https://colab.research.google.com/github/sg2e16/MANG3073A1/blob/master/MANG_3073_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##Removing the sample_data from the colaboratory files library
!rm -rf 'sample_data'

Importing:

In [0]:
##Importing the Data
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
##Importing pandas and assigning
import pandas as pd
msfd = pd.read_csv("/content/gdrive/My Drive/MANG3073/MicroSynFinal.csv")

Question 1.1:

Exploratory Data Analysis:

In [0]:
##Checking the imported Dataset
msfd.head(5)

In [0]:
##Total number of Rows and Coloumns
msfd.shape

In [0]:
msfd['Defaulter'].value_counts()

In [0]:
##Original Number of Defaulters (For use in Question 2)
ONofD = 8219
print(ONofD)

In [0]:
##Original Total of Applications (For use in Question 2)
OTofA = 50000
print(OTofA)

In [0]:
##Looking at the data types
msfd.dtypes

In [0]:
##Describing the Raw Data
msfd.describe()

Null Values identifacation and cleaning:

In [0]:
##Getting variable types and null values
msfd.info()

In [0]:
##Locating Null values
msfd.isnull()

In [0]:
##Total number of Null Values and where they are located in the headers
msfd.isnull().sum()

In [0]:
##The null values are set to "1" or the mean value
msfd.fillna(value= {'Activity':1, 'Current_Loans':1,}, inplace = True)
msfd['Collateral_valuation'].fillna((msfd['Collateral_valuation'].mean()), inplace=True)

In [0]:
##Checking that there are no remaining null values
msfd.isnull().sum()

In [0]:
##Describing the Data post null values
msfd.describe()

Identifying Outliers:

In [0]:
##Importing required packages for the graphical needs
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
##Creates pair plots for finding outliers
sns.pairplot(msfd.iloc[:,1:-1])

In [0]:
##Initial Box Plot
box = msfd.iloc[:,1:-1].boxplot(grid=False, rot=45, fontsize=13)

In [0]:
##Finding Outliers Cut = 30000
sns.distplot(msfd['Collateral_valuation'])

In [0]:
##Finding Outliers Cut = 8
sns.distplot(msfd['Properties_Total'])

In [0]:
##Finding Outliers Cut = 50000
sns.distplot(msfd['Amount'])

In [0]:
##Finding Outliers Cut = 20
sns.distplot(msfd['Historic_Loans'])

In [0]:
##Finding Outliers Cut = 7
sns.distplot(msfd['Current_Loans'])

In [0]:
##Finding Outliers Cut = 500
sns.distplot(msfd['Max_Arrears'])

In [0]:
##Trimming Variables
msfd = msfd.loc[(msfd['Collateral_valuation'] < 30000) 
                & (msfd['Properties_Total'] < 8) 
                & (msfd['Amount'] < 50000)                
                & (msfd['Historic_Loans'] < 20)
                & (msfd['Current_Loans'] < 7) 
                & (msfd['Max_Arrears'] < 500)]

In [0]:
##Post Trim Box Plot
box = msfd.iloc[:,1:-1].boxplot(grid=False, rot=45, fontsize=13)

Categorisation of the Continuous variables where it is belived to be useful:

In [0]:
##Investigating "Activity" categorical variable type
sns.distplot(msfd['Activity'])

In [0]:
##Number of times each value comes up for "Activity"
msfd['Activity'].value_counts()

In [0]:
##Grouping continuous variables to categorisations
## Codes 1-5 = A 6-11 = B 12-16 = C 17-19 = D
msfd['Activity'] = msfd['Activity'].astype(str)
msfd['Activity'] = msfd['Activity'].replace(str(1.0),'A')
msfd['Activity'] = msfd['Activity'].replace(str(2.0),'A')
msfd['Activity'] = msfd['Activity'].replace(str(3.0),'A')
msfd['Activity'] = msfd['Activity'].replace(str(4.0),'A')
msfd['Activity'] = msfd['Activity'].replace(str(5.0),'A')
msfd['Activity'] = msfd['Activity'].replace(str(6.0),'B')
msfd['Activity'] = msfd['Activity'].replace(str(7.0),'B')
msfd['Activity'] = msfd['Activity'].replace(str(8.0),'B')
msfd['Activity'] = msfd['Activity'].replace(str(9.0),'B')
msfd['Activity'] = msfd['Activity'].replace(str(10.0),'B')
msfd['Activity'] = msfd['Activity'].replace(str(11.0),'B')
msfd['Activity'] = msfd['Activity'].replace(str(12.0),'C')
msfd['Activity'] = msfd['Activity'].replace(str(14.0),'C')
msfd['Activity'] = msfd['Activity'].replace(str(15.0),'C')
msfd['Activity'] = msfd['Activity'].replace(str(16.0),'C')
msfd['Activity'] = msfd['Activity'].replace(str(17.0),'D')
msfd['Activity'] = msfd['Activity'].replace(str(18.0),'D')
msfd['Activity'] = msfd['Activity'].replace(str(19.0),'D')

In [0]:
##Number of times each value comes up for "Activity"
msfd['Activity'].value_counts()

In [0]:
##Testing to see if the changes are applied 
msfd.head(8)

Creating a Training and Test Datasets:

In [0]:
##Installing scorecardpy
!pip install scorecardpy

In [0]:
##Importing scorecardpy as sc
import scorecardpy as sc

Hotfix for bug in Scorecardpy:

In [0]:
##Hotfix: Changing "Guarantor" to a string
msfd['Guarantor'] = msfd['Guarantor'].astype(str)

In [0]:
##Hotfix: Changing "1" to "Y"
msfd['Guarantor'] = msfd['Guarantor'].replace(str(1),'Y')

In [0]:
##Hotfix: Changing "0" to "N"
msfd['Guarantor'] = msfd['Guarantor'].replace(str(0),'N')

In [0]:
##Hotfix: Changing "Collateral" to a string
msfd['Collateral'] = msfd['Collateral'].astype(str)

In [0]:
##Hotfix: Changing "1" to "Y"
msfd['Collateral'] = msfd['Collateral'].replace(str(1),'Y')

In [0]:
##Hotfix: Changing "0" to "N"
msfd['Collateral'] = msfd['Collateral'].replace(str(0),'N')

In [0]:
##Hotfix: Check to see if the values are adjusted
msfd.head(10)

Creating the train & test sets:

In [0]:
## Split in train and test
train, test = sc.split_df(msfd.iloc[:,1:],
                          y = 'Defaulter',
                          ratio = 0.7,
                          seed = 28845617).values()##Student ID as Seed

In [0]:
##The describing the train dataset
train.describe()

Coding the Nominal Variables using Weights of Evidence (WOE):

In [0]:
##Bin Variables
binValues = sc.woebin(train, y = 'Defaulter', min_perc_fine_bin=0.02, 
                 min_perc_coarse_bin=0.05, stop_limit=0.1, 
                 max_num_bin=8,method='tree')

In [0]:
##Plotting the Bin Variables
sc.woebin_plot(binValues)

In [0]:
##Manual Bin changes (Removed after first run)
##breaks_adj = sc.woebin_adj(train, "Defaulter", binValues)

In [0]:
##The breaks for woe as found by the Manual Bin
breaks_adj = {'Activity': ['C','D%,%A','B'],'Age': [33,41,48],
              'Amount':[3000,8000,22000],'Area': [25,40,55,65],
              'Collateral':['Y','N'],'Collateral_valuation':[2000,4500,13000],
              'Crrent_Loans':[3,4],'Guarantor':['Y','N'],
              'Historic_Loans':[2,4,8],'Max_Arrears':[2,24,76],
              'Properties_Status':['B','D','A%,%C'],'Properties_Total':[2,3],
              'Region': [5,8,10],'Term': [14,27,38]}

In [0]:
##Applying Bin changes
bins_adj = sc.woebin(train, y="Defaulter", breaks_list=breaks_adj) 
train_woe = sc.woebin_ply(train, bins_adj)
test_woe = sc.woebin_ply(test, bins_adj)

In [0]:
##Viewing the woe train set
train_woe.head()

IV Values Key:
- $IV < 0.02$: Remove.
- $0.02 \le IV < 0.1$: Remove.
- $0.1 \le IV < 0.3$: Keep.
- $0.3 \le IV < 1$: Keep.
- $1 \le IV $: Keep/Review. Study if error in calculation (i.e. WoE leaves a category with 100% goods or bads) or if variable is capturing future information.

In [0]:
##Look at the IV values
sc.iv(train_woe, 'Defaulter')

In [0]:
##listing column order
train_woe.columns

In [0]:
##Dropping IVs below 0.1
train_woe = train_woe.drop(['Age_woe','Properties_Status_woe','Area_woe',
                           'Collateral_woe','Max_Arrears_woe',
                           'Current_Loans_woe','Guarantor_woe'],axis = 1)
test_woe = test_woe.drop(['Age_woe','Properties_Status_woe','Area_woe',
                           'Collateral_woe','Max_Arrears_woe',
                           'Current_Loans_woe','Guarantor_woe'],axis = 1)
train_woe.head()

In [0]:
##Looking at the test woe data
test_woe.head()

Question 1.2:

Logistic Regression:

In [0]:
##Building the Logistic Regression
from sklearn.linear_model import LogisticRegression
msfd_logreg = LogisticRegression(penalty='l1',##l1 = lasso
                                     tol=0.0001,
                                     C=1.0,
                                     fit_intercept=True,
                                     class_weight='balanced',
                                     random_state=28845617,##Student ID as seed
                                     max_iter=100,
                                     verbose=1,
                                     solver = 'saga',
                                     warm_start=False)

In [0]:
##Fitting the Logistic Regression
msfd_logreg.fit(X = train_woe.iloc[:,1:], y = train_woe['Defaulter'])

In [0]:
##Parameter values
coef_msfd = pd.concat([pd.DataFrame({'column': train_woe.columns[1:]}),
                       pd.DataFrame(np.transpose(msfd_logreg.coef_))],
                      axis = 1)

coef_msfd

In [0]:
##If the intercept value is close to 0 this would prove this is a balanced 
##logistic regression that uses WoE transformation that is working as intended
msfd_logreg.intercept_

In [0]:
##Applying to the test data
pred_class_test = msfd_logreg.predict(test_woe.iloc[:, 1:])
probs_test = msfd_logreg.predict_proba(test_woe.iloc[:, 1:])
print(probs_test, pred_class_test)

Looking at ROC Curves:

In [0]:
##Importing the packages for the ROC Curves
from sklearn.metrics import roc_auc_score, confusion_matrix

In [0]:
##Showing the results of the test set after applying the model
confusion_matrix(y_true = test_woe['Defaulter'], y_pred = pred_class_test)

In [0]:
##Creating the confusion matrix for Logistic Regression
confusion_matrix_rf = confusion_matrix(y_true = test_woe['Defaulter'], 
                    y_pred = pred_class_test)

confusion_matrix_rf = confusion_matrix_rf.astype('float') / confusion_matrix_rf.sum(axis=1)[:, np.newaxis]

df_cm = pd.DataFrame(
        confusion_matrix_rf, index=['Safe', 'Default'], columns=['Safe', 'Default'],)

figsize = (10,7)
fontsize=14

fig = plt.figure(figsize=figsize)
heatmap = sns.heatmap(df_cm, annot=True, fmt='.2f')

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, 
                             ha='right', fontsize=fontsize)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45,
                             ha='right', fontsize=fontsize)

plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

In [0]:
##ROC Score
roc_auc_score(y_true = test_woe['Defaulter'], y_score = probs_test[:,1])

In [0]:
##Creating a visual ROC curve
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(test_woe['Defaulter'], probs_test[:,1])

auc = np.round(roc_auc_score(y_true = test_woe['Defaulter'], 
                             y_score = probs_test[:,1]),
              decimals = 3)

plt.plot(fpr,tpr,label="MSFD, auc="+str(auc))
plt.legend(loc=4)
plt.show()

Scorecard:

In [0]:
##Creting a Scorecard with base points = 100 base odds = 0.01 and pdo = 10
msfd_sc = sc.scorecard(bins_adj, msfd_logreg,train_woe.columns[1:],
                       points0=1000,odds0=0.01,pdo=100) 

In [0]:
##Scorecard
msfd_sc

In [0]:
##Applying the credit score over the original data
train_score = sc.scorecard_ply(train, msfd_sc, 
                               print_step=0)
test_score = sc.scorecard_ply(test, msfd_sc, 
                               print_step=0)

In [0]:
##Describing the train score
train_score.describe()

Random Forests:

In [0]:
##Creating the Random Forest Model
from sklearn.ensemble import RandomForestClassifier
msfd_rf = RandomForestClassifier(n_estimators=1000,
                       criterion='gini',
                       max_depth=None,
                       min_samples_split=2,
                       min_samples_leaf=0.001,
                       min_weight_fraction_leaf=0.0,
                       max_features='auto',
                       max_leaf_nodes=None,
                       min_impurity_decrease=0.0001,
                       bootstrap=True,
                       oob_score=True,
                       n_jobs=-1,
                       random_state=28845617, #Student ID as Seed
                       verbose=1,
                       warm_start=False,
                       class_weight='balanced')

In [0]:
##Describing the training data
train.describe()

In [0]:
##Creating a Dummy variable for "Activity"
pd.crosstab(train['Activity'],train['Defaulter'])

In [0]:
##Creating a Dummy variable for "Activity"
train.groupby('Activity', as_index=False).agg({'Defaulter': ["mean"]})

In [0]:
##Creating a Dummy variable for "Activity"
train = pd.get_dummies(train)
train.describe()

In [0]:
##Fitting the model to train set
msfd_rf.fit(train.iloc[:,:-1], train['Defaulter'])

In [0]:
# Apply the model to the test set.
rf_pred_class_test = msfd_rf.predict(train.iloc[:, :-1])
rf_probs_test = msfd_rf.predict_proba(train.iloc[:, :-1])

In [0]:
##Creating the confusion matrix for Random Forest
confusion_matrix_rf = confusion_matrix(y_true = train['Defaulter'], 
                    y_pred = rf_pred_class_test)

confusion_matrix_rf = confusion_matrix_rf.astype('float') / confusion_matrix_rf.sum(axis=1)[:, np.newaxis]

df_cm = pd.DataFrame(
        confusion_matrix_rf, index=['good', 'bad'], columns=['good', 'bad'],)

figsize = (10,7)
fontsize=14

fig = plt.figure(figsize=figsize)
heatmap = sns.heatmap(df_cm, annot=True, fmt='.2f')

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, 
                             ha='right', fontsize=fontsize)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45,
                             ha='right', fontsize=fontsize)

plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

In [0]:
##Creating the ROC curve
fpr, tpr, thresholds = roc_curve(train['Defaulter'], rf_probs_test[:,1])

auc = np.round(roc_auc_score(y_true = train['Defaulter'], 
                             y_score = rf_probs_test[:,1]),
              decimals = 3)

plt.plot(fpr,tpr,label="msfd RF, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [0]:
##The Importance of each variable
for name, importance in zip(train.columns[:-1], msfd_rf.feature_importances_):
  print(name, "= %.2f" % importance)

Random Forest WoE:

In [0]:
##Creating the Random Forest Model
from sklearn.ensemble import RandomForestClassifier
msfd_rf_woe = RandomForestClassifier(n_estimators=1000,
                       criterion='gini',
                       max_depth=None,
                       min_samples_split=2,
                       min_samples_leaf=0.001,
                       min_weight_fraction_leaf=0.0,
                       max_features='auto',
                       max_leaf_nodes=None,
                       min_impurity_decrease=0.0001,
                       bootstrap=True,
                       oob_score=True,
                       n_jobs=-1,
                       random_state=28845617, #Student ID as Seed
                       verbose=1,
                       warm_start=False,
                       class_weight='balanced')

In [0]:
##Describing the training data
train_woe.describe()

In [0]:
##Fitting the model to train set
msfd_rf_woe.fit(train_woe.iloc[:,:-1], train_woe['Defaulter'])

In [0]:
# Apply the model to the test set.
rf_pred_class_test_woe = msfd_rf_woe.predict(train_woe.iloc[:, :-1])
rf_probs_test_woe = msfd_rf_woe.predict_proba(train_woe.iloc[:, :-1])

In [0]:
##Creating the confusion matrix for Random Forest
confusion_matrix_rf_woe = confusion_matrix(y_true = train_woe['Defaulter'], 
                    y_pred = rf_pred_class_test_woe)

confusion_matrix_rf_woe = confusion_matrix_rf_woe.astype('float') / confusion_matrix_rf_woe.sum(axis=1)[:, np.newaxis]

df_cm = pd.DataFrame(
        confusion_matrix_rf, index=['good', 'bad'], columns=['good', 'bad'],)

figsize = (10,7)
fontsize=14

fig = plt.figure(figsize=figsize)
heatmap = sns.heatmap(df_cm, annot=True, fmt='.2f')

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, 
                             ha='right', fontsize=fontsize)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45,
                             ha='right', fontsize=fontsize)

plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

In [0]:
##Creating the ROC curve
fpr, tpr, thresholds = roc_curve(train_woe['Defaulter'], rf_probs_test_woe[:,1])

auc = np.round(roc_auc_score(y_true = train_woe['Defaulter'], 
                             y_score = rf_probs_test_woe[:,1]),
              decimals = 3)

plt.plot(fpr,tpr,label="msfd RF, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [0]:
##The Importance of each variable
for name, importance in zip(train.columns[:-1], msfd_rf_woe.feature_importances_):
  print(name, "= %.2f" % importance)

Question 2:

In [0]:
import scipy.stats as stats
from scipy.optimize import fsolve

In [0]:
##Calculating Average Default Rate
ONofD / OTofA

In [0]:
##Offer 1:Grant the loan and charge a fixed interest of £50 Mean Value
MV1 = 5000*(1-0.16438)
print(MV1)

In [0]:
##Offer 1:Grant the loan and charge a fixed interest of £50 Variance Value
VV1 = (5000*0.16438*(1-0.16438))
print(VV1)

In [0]:
##Offer 2:Grant the loan and charge a fixed interest of £200.
##This multiplies default rate by a factor of three. Mean Value 
MV2 = 5000 * (1-0.16438*3)
print(MV2)

In [0]:
##Offer 2: Grant the loan and charge a fixed interest of £200. 
##This multiplies default rate by a factor of three. Variance Value
VV2 = (5000*0.16438*3*(1-0.16438*3))
print(VV2)

In [0]:
##Offer 3: Grant the loan and charge a fixed interest of £500.
##This multiplies default rate by a factor of five. Mean Value
MV3 = 5000*(1-0.16438*5)
print(MV3)

In [0]:
##Offer 3: Grant the loan and charge a fixed interest of £500. 
##This multiplies default rate by a factor of five. Variance Value
VV3 = (5000*0.16438*5*(1-0.16438*5))
print(VV3)

In [0]:
P = [500,200,50]
MV = [MV3, MV2, MV1]
VV = [VV3, VV2, VV1]

In [0]:
Pavg = np.average(P[1:2], weights=MV[1:2])
Pavg

In [0]:
MVavg = np.sum(MV[0:2])
VVavg = np.sum(VV[0:2])
print('MVavg = ', MVavg)
print('VVavg = ', VVavg)

EMSR-A

In [0]:
fpnormal = stats.norm(loc = MV[0], scale = np.sqrt(VV[0]))
def flittle(y, p1 = P[0], p2 = P[1]):
  out = 1-fpnormal.cdf(y) - p2/p1
  return(out)

fsolve(flittle,900)

In [0]:
fpnormal2 = stats.norm(loc = MV[1], scale = np.sqrt(VV[1]))
def flittle(y, p1 = P[1], p2 = P[2]):
  out = 1-fpnormal2.cdf(y) - p2/p1
  return(out)

fsolve(flittle,2800)

Number of Loans: Offer 3 = 897,  Offer 2 = 1661, Offer 1 = 1942

EMSR-B

In [0]:
normal = stats.norm(loc = MV[0], scale = np.sqrt(VV[0]))
def flittle(y, p1 = P[0], p2 = P[1]):
    out = 1- normal.cdf(y) - p2/p1
    return(out)

fsolve(flittle,900)

In [0]:
normal = stats.norm(loc = MVavg, scale = np.sqrt(VVavg))
def flittle(y, p1 = Pavg, p2 = P[2]):
    out = 1 - normal.cdf(y) - p2/p1
    return(out)

fsolve(flittle, 3700)

Number of Loans: Offer 3 = 897,  Offer 2 = 2557, Offer 1 = 1046